In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator

SEED = 42
BATCH_SIZE = 32
TARGET_HEIGHT = 150
TARGET_WIDTH = 150

np.random.seed(SEED)

c:\users\yangj\.conda\envs\garbage_app\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\yangj\.conda\envs\garbage_app\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\yangj\.conda\envs\garbage_app\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\yangj\.conda\envs\garbage_app\lib\site-packages\tensorf

In [2]:
cur_dir = os.getcwd()
dataset_path = "{}/dataset".format(cur_dir)
labels_path = "{}/labels.csv".format(dataset_path)

# load labels into a dataframe
labels_df = pd.read_csv(labels_path)
labels_df.head()


,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [3]:
# check for empty cells
assert not labels_df.isnull().values.any()


In [4]:
# check for the number of breeds in the dataset
print(labels_df.breed.nunique())


120


In [5]:
# split train and validation sets
train_folder = "{}/train".format(dataset_path)
labels_df["paths"] = labels_df.apply(
    lambda row: ("{}/{}.jpg".format(train_folder, row["id"])), axis=1
)

training_data = np.array([img_to_array(load_img(path, target_size=(TARGET_HEIGHT, TARGET_WIDTH)))
                          for path in labels_df["paths"].values.tolist()] 
                         ).astype("float32")
target_breeds = np.array(labels_df.breed)

x_train, x_test, y_train, y_test = train_test_split(
    training_data, target_breeds, test_size=0.2, stratify=target_breeds, random_state=SEED
)

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, stratify=np.array(y_train), random_state=SEED
)


In [7]:
# Convert labels to one hot encoding
y_train_ohe = pd.get_dummies(pd.Series(y_train).reset_index(
    drop=True
)).values

y_val_ohe = pd.get_dummies(pd.Series(y_val).reset_index(
    drop=True
)).values

y_test_ohe = pd.get_dummies(pd.Series(y_test).reset_index(
    drop=True
)).values


In [8]:
""" Data Augmentation """

# for training set
train_img_generator = ImageDataGenerator(
    rescale=1./255, rotation_range=30, width_shift_range=0.2,
    height_shift_range=0.2, horizontal_flip=True
)

train_generator = train_img_generator.flow(
    x_train, y_train_ohe, shuffle=True, batch_size=BATCH_SIZE, seed=1
)

# for validation set
val_img_generator = ImageDataGenerator(rescale=1./255)

val_generator = val_img_generator.flow(
    x_val, y_val_ohe, shuffle=True, batch_size=BATCH_SIZE, seed=1
)


In [ ]:
# Classifier

inception_V3 = tf.keras.applications.InceptionV3(weights="imagenet", include_top=False, input_shape=(TARGET_HEIGHT, TARGET_WIDTH, 3))

out = inception_V3.output
out = tf.keras.layers.GlobalAveragePooling2D()(out)
out = tf.keras.layers.Dense(512, activation="relu")(out)
out = tf.keras.layers.Dense(512, activation="relu")(out)
total_classes = y_train_ohe.shape[1]

predictions = tf.keras.layers.Dense(total_classes, activation="softmax")(out)

model = tf.keras.models.Model(inputs=inception_V3.input, outputs=predictions)

# make the inception v3 layers non trainable (only train the extra layers)
for layer in inception_V3.layers:
    layer.trainable = False
    
steps_per_epoch = x_train.shape[0]
validation_steps = x_val.shape[0]
epochs = 15
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
history = model.fit_generator(
    train_generator, steps_per_epoch=steps_per_epoch,
    validation_data=val_generator, validation_steps=validation_steps,
    epochs=epochs, verbose=1
)

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("model.h5")


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/15
  75/7359 [..............................] - ETA: 3:09:29 - loss: 4.7985 - acc: 0.0117